<a href="https://colab.research.google.com/github/NBackshall/machine-learning-learning/blob/main/Automatic_Differentiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic differentiation
Automatic differentiation (AD), also called algorithmic differentiation or simply "autodiff", is a family of techniques similar to but more general than backpropagation for efficiently and accurately evaluating derivatives of numeric functions expressed as computer programs.

Autodiff is a solution to the problems encountered with Numerical Differentiation (which has a complxity of $O(n)$) and Symbolic Differentiation (which can suffer from expression swell where $f'(x)$ may be exponentially longer than $f(x)$ caused by rules like the product rule; also, Symbolic requires closed-form expressions - this limits ability to use control flow mechanisms: conditionals, loops and recursion). Autodiff operates directly on program of interest (whilst achieving the same accuracy as symbolic), rather than obtaining an expression the goal of autodiff is to calculate its numerical value.

In [21]:
import math

AD exploits the fact that every computer program, no matter how complicated, executes a sequence of elementary arithmetic operations (addition, subtraction, multiplication, division, etc.) and elementary functions (exp, log, sin, cos, etc.).

Additionally, AD utilises the decompositin of differentials provided by the chain rule:

$y=f(g(h(x)))=f(g(h(w_0)))=f(g(w_1))=f(w_2)=w_3$

$w_0=x$

$w_1=h(w_o)$

$w_2=g(w_1)$

$w_3=f(w_2)=y$

Using the above, the chain rule gives:

$\frac{dy}{dx}=\frac{dy}{dw_2}\frac{dw_2}{dw_1}\frac{dw_1}{dx}$

$\frac{dy}{dx}=\frac{df(w_2)}{dw_2}\frac{dg(w_1)}{dw_1}\frac{dh(w_0)}{dw_0}$

Forward accumulation (or mode) and backward accumulation (or mode) changes which direction you travers this chain rule (forward: left to right; backward: right to left).

1.   Forward mode: $\frac{dw_i}{dx}=\frac{dw_i}{dw_{i-1}}\frac{dw_{i-1}}{dx}$ (for above example $w_3=y$)
2.   Backward mode: $\frac{dy}{dw_i}=\frac{dy}{dw_{i+1}}\frac{dw_{i+1}}{dw_i}$ (for above example $w_0=x$)

## Forward mode
In forward mode AD, one first fixes the independent variable w.r.t. which differentiation is performed and computes the derivative of each sub-expression recursively. This involves repeatedly substituting the derivative of the inner functions in the chain rule:

$\frac{∂y}{∂x}=\frac{∂y}{∂w_{n-1}}\frac{∂w_{n-1}}{∂x}$

$\frac{∂y}{∂x}=\frac{∂y}{∂w_{n-1}}(\frac{∂w_{n-1}}{∂w_{n-2}}\frac{∂w_{n-2}}{∂x})$ and so on...

Forward mode is easier to implement as the each derivitive is calculated before it is required. Each variable $w$ is stored in a tuple with it's derivitive, $ẇ=\frac{∂w}{∂x}$.

As an example:

$y=f(x_1,x_2)$

$y=x_1x_2+sin(x_1)$

$y=w_1w_2+sin(w_1)$

$y=w_3+w_4$

$y=w_5$

The choice of the independent variable to which differentiation is performed affects the seed values $\dot{w}_1$ and $\dot{w}_2$. Given interest in the derivative of this function with respect to $x_1$, the seed values should be set to:

$ẇ_1=\frac{∂x_1}{∂x_1}=1$


$\dot{w}_2=\frac{∂x_2}{∂x_1}=0$

With these seed values:

$(w_1,\dot{w}_1)=(w_1,1)$

$(w_2,\dot{w}_2)=(w_2,0)$

$(w_3,\dot{w}_3)=(w_1w_2,w_1\dot{w_2}+\dot{w_1}w_2)$

$(w_4,\dot{w}_4)=(sin(x_1),sin(x_1)\dot{w}_1)$

$(w_5,\dot{w}_5)=(w_3+w_4,\dot{w_3}+\dot{w_4})$

In this case, to the gradient requires computing derivitive w.r.t. $x_1$ and $x_2$; therefore, two sweeps are required switching the seed falues from $[1,0]$ to $[0,1]$.

# Dual numbers
In order to implment Forward mode AD dual numbers can be used with operation overloading. This is shown above where each variable $w$ is augmented with its derivitive, $\langle{w,\dot{w}}\rangle{}$.

The new arithmetic consists of ordered pairs, elements written $\langle x, \dot{x} \rangle$, with ordinary arithmetics on the first component, and first order differentiation arithmetic on the second component, as described above. For instance, multiplication:

$\langle{u,\dot{u}}\rangle{}\times\langle{v,\dot{v}}\rangle{}=\langle{uv,u\dot{v}+v\dot{u}}\rangle{}$

And also $sin$:

$sin\langle{u,\dot{u}}\rangle{}=\langle{sin(u),\dot{u}\times{}cos(u)}\rangle{}$

In [23]:
class DualNumber:
  def __init__(self, value, dvalue):
    self.value = value
    self.dvalue = dvalue

  def __str__(self):
    return str(self.value) + " + " + str(self.dvalue) + "ε"

  def __add__(self, other):
    return DualNumber(
        self.value + other.value,
        self.dvalue + other.dvalue
    )

  def __sub__(self, other):
    return DualNumber(
        self.value - other.value,
        self.dvalue - other.dvalue
    )

  def __mul__(self, other):
    return DualNumber(
        self.value * other.value,
        self.value * other.dvalue + self.dvalue * other.value
    )

  def __truediv__(self,other):
    return DualNumber(
        self.value / other.value,
        (other.value * self.dvalue - self.value * other.dvalue) / other.value**2
    )
  
  def __pow__(self, other):
    value = self.value ** other.value
    dvalue = value * (self.dvalue * (other.value / self.value) + other.dvalue * math.log(self.value))
    return DualNumber(value,dvalue)
    

Testing Dual number using two equations: $u=x^2$ and $v=5x$

In [24]:
x = 1.2
u = DualNumber(x**2,2*x)
v = DualNumber(5*x,5)

test_value = ((x**2)**(5*x) + (x**2)*(5*x)) / ((x**2) - (5*x))
test_dvalue = (15*x**2 + (x**2)**(5*x) * (5*math.log(x**2) + 10)) / (
    x**2 - 5*x) - ((2*x - 5) * (5*x**3 + (x**2)**(5*x))) / (x**2 - 5*x)**2
dual_number = (u**v + u*v) / (u - v)

print("The correct answer is {} + {}ε \nDualNumber found      {}".format(test_value, test_dvalue, dual_number))

The correct answer is -3.8500220281263147 + -25.659412357895754ε 
DualNumber found      -3.8500220281263147 + -25.65941235789575ε


Next we need to be able to handle mathematical functions such as sin and exp.

In [25]:
def sin(x: DualNumber):
  return DualNumber(math.sin(x.value), x.dvalue * math.cos(x.value))

def cos(x: DualNumber):
  return DualNumber(math.cos(x.value), - x.dvalue * math.sin(x.value))

def exp(x: DualNumber):
  return DualNumber(math.exp(x.value), x.dvalue * math.exp(x.value))

Testing for $y=f(x)=sin^{cos(x^3)}(2x)$ where $x=1.2$ (from WolframAlpha.com):

$f(1.2)=1.06335$

$f'(1.2)=2.14361$

In [26]:
x = 1.2
w_1 = DualNumber(x,1)
w_2 = DualNumber(2,0)*w_1
w_3 = w_1**DualNumber(3,0)
w_4 = sin(w_2)
w_5 = cos(w_3)
w_6 = w_4**w_5

print("w_1 = {}\nw_2 = {}\nw_3 = {}\nw_4 = {}\nw_5 = {}\nw_6 = {}, ".format(w_1,w_2,w_3,w_4,w_5,w_6))

w_1 = 1.2 + 1ε
w_2 = 2.4 + 2.0ε
w_3 = 1.7279999999999998 + 4.319999999999999ε
w_4 = 0.675463180551151 + -1.4747874310824909ε
w_5 = -0.15655697721737202 + -4.266729772345185ε
w_6 = 1.0633519842136765 + 2.1436132251798043ε, 


Testing for $y=f(x)=e^{sin^{cos(x^3)}(2x)}$ where $x=1.2$ (from WolframAlpha.com):

$f(1.2)=2.89606$

$f'(1.2)=6.20804$

In [27]:
w_7 = exp(w_6)
print("w_7 = {}".format(w_7))

w_7 = 2.8960622927327844 + 6.2080374316465425ε


## Backward mode

A really helpful article: https://rufflewind.com/2016-12-30/reverse-mode-automatic-differentiation

The implementation simplicity of forward-mode AD comes with a big disadvantage, which becomes evident when we want to calculate both ∂z/∂x and ∂z/∂y. In forward-mode AD, doing so requires seeding with dx = 1 and dy = 0, running the program, then seeding with dx = 0 and dy = 1 and running the program again. In effect, the cost of the method scales linearly as O(n) where n is the number of input variables. This would be very costly if we wanted to calculate the gradient of a large complicated function of many variables, which happens surprisingly often in practice.

One way is to parse the original program and then generate an adjoint program that calculates the derivatives. However this can be complex to implement, especially with certain programming languages.

A simpler way is to do this dynamically: construct a full graph that represents our original expression as the program runs. The goal is to get something akin to a dependency graph.

By default, a node is created without any children. However, whenever a new expression u is built out of existing nodes $w_i$, the new expression $u$ registers itself as a child of each of its dependencies $w_i$. During the child registration, it will also save its contributing weight $\frac{∂w_i}{∂u}$ which will be used later to compute the gradients. 


In [60]:
class Var:
  def __init__(self, value):
    self.value = value
    self.children = [] # [(weight, Var)]
    self.grad_value = None # None used to check if grad_value calculated

  def gradient(self):
    # recurse iff grad_value == None
    if self.grad_value is None:
      self.grad_value = sum(weight * var.gradient() for weight, var in self.children)
    return self.grad_value

  def set_gradient(self, value):
    self.grad_value = value

  def __add__(self, other):
    z = Var(self.value + other.value)
    self.children.append((1, z))
    other.children.append((1, z))
    return z

  def __sub__(self, other):
    z = Var(self.value - other.value)
    self.children.append((1, z))
    other.children.append((-1, z))
    return z

  def __mul__(self, other):
    # the weight is what the child's gradient is multiplied by as a contribution
    # to the sum used to calculate the parent gradient.
    z = Var(self.value * other.value) # z is the new child
    self.children.append((other.value, z)) # weight = ∂z/∂self = other.value
    other.children.append((self.value, z)) # weight = ∂z/∂other = self.value
    return z

  def __truediv__(self, other):
    z = Var(self.value / other.value)
    self.children.append((1/other.value, z))
    other.children.append((-self.value / other.value**2, z))
    return z

  def __pow__(self, other):
    z = Var(self.value ** other.value)
    self.children.append((self.value ** (other.value - 1), z))
    other.children.append((self.value ** other.value * math.log(self.value), z))
    return z

  def __str__(self):
    if self.grad_value is None:
      return "value={}, {} child(ren), grad_val={}".format(self.value, len(self.children), "None")
    else:
      return "value={}, {} child(ren), grad_val={}".format(self.value, len(self.children), self.grad_value)

In [61]:
u_1 = Var(0.5)
u_2 = Var(4.2)
u_3 = u_1 / u_2
u_4 = u_1 ** u_2
u_5 = u_3 + u_4


print("u_1 = {}\nu_2 = {}\nu_3 = {}\nu_4 = {}\nu_5 = {}".format(u_1,u_2,u_3,u_4,u_5))

u_5.set_gradient(1)

print("---")

print("u_1 = {}\nu_2 = {}\nu_3 = {}\nu_4 = {}\nu_5 = {}".format(u_1,u_2,u_3,u_4,u_5))

u_1 = value=0.5, 2 child(ren), grad_val=None
u_2 = value=4.2, 2 child(ren), grad_val=None
u_3 = value=0.11904761904761904, 1 child(ren), grad_val=None
u_4 = value=0.05440941020600775, 1 child(ren), grad_val=None
u_5 = value=0.1734570292536268, 0 child(ren), grad_val=None
---
u_1 = value=0.5, 2 child(ren), grad_val=None
u_2 = value=4.2, 2 child(ren), grad_val=None
u_3 = value=0.11904761904761904, 1 child(ren), grad_val=None
u_4 = value=0.05440941020600775, 1 child(ren), grad_val=None
u_5 = value=0.1734570292536268, 0 child(ren), grad_val=1


Add some mathematical function like in Forward Mode AD.

In [62]:
def b_sin(x):
  z = Var(math.sin(x.value))
  x.children.append((math.cos(x.value), z))
  return z

def b_cos(x):
  z = Var(math.cos(x.value))
  x.children.append((-math.sin(x.value), z))
  return z

def b_exp(x):
  z = Var(math.exp(x.value))
  x.children.append((math.exp(x.value), z))
  return z

To test, following example from the following forum accepted answer: https://stats.stackexchange.com/questions/224140/step-by-step-example-of-reverse-mode-automatic-differentiation

In [65]:
x = 2
y = 3
w_1 = DualNumber(x,1)
w_2 = DualNumber(y,1)
w_3 = w_1 * w_2
w_4 = sin(w_1)
w_5 = w_3 + w_4

print("w_1 = {}\nw_2 = {}\nw_3 = {}\nw_4 = {}\nw_5 = {}".format(w_1,w_2,w_3,w_4,w_5))

print("---")

u_1 = Var(w_1.value)
u_2 = Var(w_2.value)
u_3 = u_1 * u_2
u_4 = b_sin(u_1)
u_5 = u_3 + u_4


print("u_1 = {}\nu_2 = {}\nu_3 = {}\nu_4 = {}\nu_5 = {}".format(u_1,u_2,u_3,u_4,u_5))

# set gradient of dz/dz
u_5.set_gradient(1)

u_1.gradient() # calculate gradient of dz/dx
u_2.gradient() # calculate gradient of dz/dy

print("---")

print("u_1 = {}\nu_2 = {}\nu_3 = {}\nu_4 = {}\nu_5 = {}".format(u_1,u_2,u_3,u_4,u_5))

print("---")

print("dz/dx = {}".format(u_1.gradient()))
print("dz/dy = {}".format(u_2.gradient()))

w_1 = 2 + 1ε
w_2 = 3 + 1ε
w_3 = 6 + 5ε
w_4 = 0.9092974268256817 + -0.4161468365471424ε
w_5 = 6.909297426825682 + 4.583853163452858ε
---
u_1 = value=2, 2 child(ren), grad_val=None
u_2 = value=3, 1 child(ren), grad_val=None
u_3 = value=6, 1 child(ren), grad_val=None
u_4 = value=0.9092974268256817, 1 child(ren), grad_val=None
u_5 = value=6.909297426825682, 0 child(ren), grad_val=None
---
u_1 = value=2, 2 child(ren), grad_val=2.5838531634528574
u_2 = value=3, 1 child(ren), grad_val=2
u_3 = value=6, 1 child(ren), grad_val=1
u_4 = value=0.9092974268256817, 1 child(ren), grad_val=1
u_5 = value=6.909297426825682, 0 child(ren), grad_val=1
---
dz/dx = 2.5838531634528574
dz/dy = 2
